In [488]:
#dependencies
import pandas as pd
import requests

In [495]:
#read in geoJSON and state codes
states = pd.read_json('./static/geoJson/us_states.json')
state_codes = pd.read_csv('./CSVs/state_codes.csv')

In [496]:
#Drop territories
state_list = state_codes[['State/District','Postal Code']][:51]

counter = 0

#Add state codes to geoJSON for easier lookups
for state in state_list.iterrows():
    state_code = state_list['Postal Code'][counter]
    states['features'][counter]['properties']['state_code'] = state_code
    counter = counter + 1
    
#PR gets a code so the program doesn't bomb later
states['features'][51]['properties']['state_code'] = 'PR'

In [497]:
#JSON requests to api endpoint to get attributes to write to geoJSON
state_count15 = requests.get('http://localhost:5000/state_count15')
state_metal15 = requests.get('http://localhost:5000/state_metal15')
state_issuer15 = requests.get('http://localhost:5000/state_issuer15')
state_type15 = requests.get('http://localhost:5000/state_type15')

In [498]:
#read the JSONs
state_count15 = state_count15.json()
state_metal15 = state_metal15.json()
state_issuer15 = state_issuer15.json()
state_type15 = state_type15.json()

In [500]:
for dicts in state_count15: 
    for keys in dicts: 
        dicts['Count'] = int(dicts['Count']) 

In [501]:
#lookup the states that we have data for count data to the geoJSON

counter1=0
counter2=0

while counter1 < len(state_count15):
    lookup_state = state_count15[counter1]['State']
    count_state = states['features'][counter2]['properties']['state_code']
    counter2 = counter2 +1
    if lookup_state == count_state:
        states['features'][counter2]['properties']['state_count15'] = state_count15[counter1]['Count']
        counter2 = 0
        counter1 = counter1 +1

In [471]:
#break metal up into each type

year_var = metal_count15
source_var = state_metal15

year_data = pd.DataFrame(source_var)
bronze15=year_data.loc[year_data['Metal'] == 'Bronze'].reset_index(drop=True)
silver15=year_data.loc[year_data['Metal'] == 'Silver'].reset_index(drop=True)
gold15=year_data.loc[year_data['Metal'] == 'Gold'].reset_index(drop=True)
platinum15=year_data.loc[year_data['Metal'] == 'Platinum'].reset_index(drop=True)

In [472]:
#write metal data to geoJSON

counter1=0
counter2=0

pointer = bronze15

while counter1 < len(pointer):
    lookup_state = pointer['State'][counter1]
    count_state = states['features'][counter2]['properties']['state_code']
    counter2 = counter2 +1
    if lookup_state == count_state:
        states['features'][counter2]['properties'][f'{pointer}'] = pointer.loc[pointer['State'] == lookup_state]['Count'][counter1]
        counter2 = 0
        counter1 = counter1 +1


In [502]:
states['features'][2]['properties']

{'name': 'Arizona', 'density': 57.05, 'state_code': 'AZ', 'state_count15': 957}

In [503]:
states.to_json('./static/geoJson/modifiedStates.json')